In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
train =pd.read_csv("train_play.csv",parse_dates=['time'])
train.head()

,row_id,time,x,y,direction,congestion
0,0,1991-04-01,0,0,EB,70
1,1,1991-04-01,0,0,NB,49
2,2,1991-04-01,0,0,SB,24
3,3,1991-04-01,0,1,EB,18
4,4,1991-04-01,0,1,NB,60


In [5]:
test =pd.read_csv("test_play.csv",parse_dates=['time'])
test.head()


,row_id,time,x,y,direction
0,848835,1991-09-30 12:00:00,0,0,EB
1,848836,1991-09-30 12:00:00,0,0,NB
2,848837,1991-09-30 12:00:00,0,0,SB
3,848838,1991-09-30 12:00:00,0,1,EB
4,848839,1991-09-30 12:00:00,0,1,NB


In [6]:
train['month'] = train['time'].dt.month
train['day'] = train['time'].dt.day
train['hour'] = train['time'].dt.hour
train['weekday'] = train['time'].dt.weekday

test['month'] = test['time'].dt.month
test['day'] = test['time'].dt.day
test['hour'] = test['time'].dt.hour
test['weekday'] = test['time'].dt.weekday



In [7]:
le = LabelEncoder()

dir_en=pd.DataFrame(le.fit_transform(train['direction']))
X_train = train.drop(['time', 'congestion','direction'], axis=1)
X_train['direction']=dir_en

y_train = np.log1p(train['congestion'])



In [6]:
d_en=pd.DataFrame(le.fit_transform(test['direction']))
X_test =  test.drop(['time','row_id','direction'], axis=1)
X_test['direction'] = dir_en

print(X_train.columns)
print(X_test.columns)

Index(['x', 'y', 'month', 'day', 'hour', 'weekday', 'direction'], dtype='object')
Index(['x', 'y', 'month', 'day', 'hour', 'weekday', 'direction'], dtype='object')


In [7]:
X_train['weekday'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
#Linear regression

lr = LinearRegression()
#lr.fit(X_train,y_train)


kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {}
gcv = GridSearchCV(lr, param_grid = params, cv = kfold, scoring = 'neg_mean_absolute_error')
gcv.fit(X_train,y_train)
print(gcv.best_score_)
print(gcv.best_params_)

y_pred = np.expm1(gcv.predict(X_test))
y_pred

In [ ]:
submit = pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit

In [ ]:
#submit.to_csv('tabular_playground_log_linear.csv',index=False)

In [ ]:
#using elasticnet

el = ElasticNet(alpha=0.4, l1_ratio=0.2)
el.fit(X_train,y_train)

kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {}
gcv = GridSearchCV(el, param_grid = params, cv = kfold, scoring = 'neg_mean_absolute_error')
#gcv.fit(X_train,y_train)
#print(gcv.best_score_)
#print(gcv.best_params_)

y_pred = np.expm1(el.predict(X_test))
y_pred

In [ ]:
submit=pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit

In [ ]:
#submit.to_csv('tabular_playground_log_elastic.csv',index=False)

In [8]:
#using lightGBM

lgb=LGBMRegressor()

#lgb.fit()

#lgb.fit(X_train,y_train, eval_metric="mae")
kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {'learning_rate' : np.linspace(0.001, 0.9, 10), 'max_depth' : [None, 3, 4], 'n_estimators' : [25, 50, 100]}
gcv1 = GridSearchCV(lgb, param_grid = params, cv = kfold, scoring = 'neg_root_mean_squared_error')
gcv1.fit(X_train, y_train)
print(gcv1.best_score_)
print(gcv1.best_params_)

y_pred=np.expm1(gcv1.predict(X_test))
y_pred

C:\ProgramData\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\ProgramData\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011565 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 679068, number of used features: 7
[LightGBM] [Info] Start training from score 3.815684
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 679068, number of used features: 7
[LightGBM] [Info] Start training from score 3.816173
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011671 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, y

array([46.44735711, 36.31965403, 42.57456268, ..., 62.63149169,
       14.80212086, 37.99471918])

In [9]:
submit=pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit

,row_id,congestion
0,848835,46.447357
1,848836,36.319654
2,848837,42.574563
3,848838,25.107566
4,848839,69.597184
...,...,...
2335,851170,58.361161
2336,851171,30.678595
2337,851172,62.631492
2338,851173,14.802121


In [10]:
#submit.to_csv('Tabular_Playground_log_lgbm.csv',index=False)

In [ ]:
#using xgb
xgb = XGBRegressor(random_state=24)
xgb.fit(X_train,y_train)

kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {'learning_rate' : np.linspace(0.001, 0.9, 10), 'max_depth' : [None, 3, 4], 'n_estimators' : [25, 50, 100]}
gcv1 = GridSearchCV(xgb, param_grid = params, cv = kfold, scoring = 'neg_root_mean_squared_error')
gcv1.fit(X_train, y_train)
print(gcv1.best_score_)
print(gcv1.best_params_)

y_pred=np.expm1(gcv1.predict(X_test))
y_pred

In [ ]:
submit=pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit

In [ ]:
#submit.to_csv('Tabular_Playground_log_XGB.csv',index=False)

In [ ]:
#using cat boost

cat = CatBoostRegressor(random_state=24)
cat.fit(X_train, y_train)

y_pred=np.expm1(cat.predict(X_test))


In [ ]:
submit=pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit

In [ ]:
#submit.to_csv('Tabular_Playground_log_cat.csv',index=False)